In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from pathlib import Path

from time import sleep

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.utils import shuffle

from importlib import reload
import sentinel_utils
import keras_model_creator

from data_generator import DataGenerator

In [2]:
sentinel_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12']
soilgrids_band = ['bdod', 'cec', 'cfvo', 'clay', 'nitrogen', 'ocd',
                  'ocs', 'phh2o', 'sand', 'silt', 'soc']
all_bands = sentinel_bands + ['Elevation'] + soilgrids_band
selected_bands = all_bands
bands = [all_bands.index(b) for b in selected_bands]
bands

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [3]:
reload(sentinel_utils)

shards_dir = Path.home().joinpath('sentinel_data').joinpath('shards')

# sort_key = lambda x: int(x.stem.split('_')[-1])
sentinel_shards = []
seasons = ['03', '06', '09']
year = 2017
for s in seasons:
    path_list = list(shards_dir.joinpath(f'features_{year}{s}').glob('feature_*.npy'))
    sentinel_shards.extend(path_list)

all_labels = pd.read_csv(Path('data').joinpath('full_dummies.csv'))
sample_shards = 20000
utils = sentinel_utils.SentinelUtils(
    all_labels, all_bands,
    shuffle(sentinel_shards, random_state=42)[:sample_shards],
    min_occurrences=5000, overwrite_existing=False
)
shard_ids = utils.selected_classes.index

In [4]:
loss = 'binary_focal_crossentropy'
batch_size = 32
tag = 'conv3d+lstm'

model_dir = Path('models').joinpath(
    f'{loss}-{len(shard_ids)}-{utils.selected_classes.shape[1]}'
    f'-{len(bands)}-{year}-{"_".join(seasons)}-{batch_size}-{tag}'
)
model_dir.mkdir(parents=True, exist_ok=True)
model_dir

PosixPath('models/binary_focal_crossentropy-237797-30-22-2017-03_06_09-32-conv3d+lstm')

In [ ]:
reload(keras_model_creator)
params = dict(
    utils=utils,
    shards_dir=shards_dir,
    shard_ids=shard_ids,
    model_dir=model_dir,
    seasons=seasons,
    year=year,
    bands=bands,
    loss=loss,
    batch_size=batch_size,
    base_filters=32,
    dropout=0.2,
    epochs=10,
    overwrite=False
)

model, testing_generator = keras_model_creator.KerasModelCreator(**params).run()

Building model...
Fitting...
Epoch 1/10


I0000 00:00:1720444224.899596   12147 service.cc:145] XLA service 0x7f41fc047fb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720444224.899665   12147 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1720444250.425144   12147 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7118/7118 ━━━━━━━━━━━━━━━━━━━━ 2201s 305ms/step - accuracy: 0.2388 - auc: 0.8509 - loss: 0.0592 - macrof1score: 0.1665 - microf1score: 0.3234 - prc: 0.3622 - precision: 0.5024 - recall: 0.2385 - weightedf1score: 0.2922 - val_accuracy: 0.3520 - val_auc: 0.9382 - val_loss: 0.0380 - val_macrof1score: 0.2217 - val_microf1score: 0.4458 - val_prc: 0.6086 - val_precision: 0.7688 - val_recall: 0.3139 - val_weightedf1score: 0.3779 - learning_rate: 0.0010
Epoch 2/10
7118/7118 ━━━━━━━━━━━━━━━━━━━━ 2266s 318ms/step - accuracy: 0.3382 - auc: 0.9258 - loss: 0.0406 - macrof1score: 0.2438 - microf1score: 0.4485 - prc: 0.5530 - precision: 0.6817 - recall: 0.3342 - weightedf1score: 0.3916 - val_accuracy: 0.3648 - val_auc: 0.9447 - val_loss: 0.0359 - val_macrof1score: 0.2653 - val_microf1score: 0.5039 - val_prc: 0.6382 - val_precision: 0.7556 - val_recall: 0.3780 - val_weightedf1score: 0.4206 - learning_rate: 0.0010
Epoch 3/10
3537/7118 ━━━━━━━━━━━━━━━━━━━━ 39:24 660ms/step - accuracy: 0.3513 - auc: 0.93

In [ ]:
# for eval_year in [2020, 2021]:
#     params['year'] = eval_year
#     eval_generator = DataGenerator(shard_ids, shuffle=False, **params)
#     model.evaluate(x=eval_generator, verbose=1, return_dict=True)
    
#     eval_path = model_dir.joinpath(f'eval_{eval_year}.csv')
#     if eval_path.is_file():
#         print(pd.read_csv(eval_path))
#     else:
#         r = model.evaluate(x=testing_generator, verbose=1, return_dict=True)
#         df = pd.DataFrame.from_dict(r, orient='index', columns=['score'])
#         df.index.name = 'metric'
#         df.to_csv(eval_path)
#         print(df)

In [ ]:
# for eval_year in [2020, 2021]:
# params['year'] = eval_year
eval_generator = DataGenerator(shard_ids, shuffle=False, **params)
preds_path = model_dir.joinpath(f"preds_{params['year']}.npy")
if preds_path.is_file():
    y_pred = np.load(preds_path)
else:
    y_pred = model.predict(x=eval_generator, verbose=1)
    np.save(preds_path, y_pred)

In [ ]:
y_true = utils.selected_classes.iloc[:-(utils.selected_classes.shape[0]%batch_size)].to_numpy()
cm = confusion_matrix(y_true.flatten(), (y_pred > 0.5).flatten().astype(int))
ConfusionMatrixDisplay(confusion_matrix=cm).plot()

In [ ]:
import matplotlib.pyplot as plt

class_names = utils.selected_classes.columns

f, axes = plt.subplots(6, 5, figsize=(25, 30))
axes = axes.ravel()
for label in range(y_true.shape[1]):
    cm = confusion_matrix(y_true[..., label].astype(int), (y_pred[..., label] > 0.5).astype(int))
    disp = ConfusionMatrixDisplay(cm)
    disp.plot(ax=axes[label], values_format='.4g')
    disp.ax_.set_title(f'{class_names[label]}')
    if label < 25:
        disp.ax_.set_xlabel('')
    if label % 5 != 0:
        disp.ax_.set_ylabel('')
    disp.im_.colorbar.remove()

plt.subplots_adjust(wspace=0.2, hspace=0.001)
f.colorbar(disp.im_, ax=axes)
plt.show()


In [ ]:
import tensorflow.keras.backend as K
K.eval(model.optimizer.learning_rate)

In [ ]:
# tf.keras.utils.plot_model(model, show_shapes=True)
# model.summary()

In [ ]:
# import subprocess
# subprocess.run(['sudo', 'shutdown', 'now'])